# Project 1: Reading comprehension



# Introduction

Tutorial: https://lightning.ai/docs/pytorch/stable/starter/introduction.html#

W&B Link: TODO

# Setup

## Dependencies
Install all necessary dependencies
- PyTorch: `torch lightning`
- Hugging Face: `huggingface_hub datasets`
- Weights & Biases: `wandb`
- nltk: `nltk`

Optional
- Lint and Formatting: `ruff`

Dependencies are pinned to the version the code was created with. 

## Notebook setup
Log into Hugging Face and Weights & Biases.

## Tools used
- Visual Studio Code
- GitHub Copilot

In [ ]:
%pip install torch huggingface_hub datasets wandb ruff

In [ ]:
from huggingface_hub import login

login()

In [ ]:
import wandb

wandb.login()

# Preprocessing

Predefined requirements:
- Train / Validation / Test split
- Existing word embedding model: word2vec, GloVe, fastText

Download the BoolQ dataset with `datasets` and split it in the predefined way.

Data treatment steps:
- Tokenize
- Lower case
- Stop word removal
- lemmatization 
- truncating of passage (enforce maximum length, ~99% not truncated)
- Embedding with fasttext

Used features: `question`, `answer`, `passage` (all of them)

Input format: `question` and `passage` vectors
Label format: `answer` 1 or 0

Batching: None, dataset is small enough (6.5k rows in train)

Correctness tests:
Check texts before embedding if they still make sense 

In [ ]:
from datasets import load_dataset

train = load_dataset("google/boolq", split="train[:-1000]")
valid = load_dataset("google/boolq", split="train[-1000:]")
test = load_dataset("google/boolq", split="validation")

print(len(train), len(valid), len(test))

# Model

Predefined requirements:
- Classifier
    - 2 Layers
    - ReLu

network architecture
- input layer
    - dim: max len of input x 2
- 2 hidden layers
    - 0.5 * max x 2
    - ReLu activation
- output layer
    - output of class (1 true, 0 false)
        - dim: 1x1
        - not probability output because the question should be answered with yes or no, not 60% yes
    - sigmoid activation
- normalization: done in preprocessing
- regularization: done by optimizer

Loss function either:
- HingeEmbeddingLoss: measures whether two inputs are similar or not
- Binary Cross-Entropy: separate to classes
Optimizer either:
- Adam: good default choice
- AdamW: supposed improvements to Adam

Experiments:
- Different Loss and Optimizer combiniations
- Size of input & hidden layers
- Epochs, Learning rate 

Checkpoints: every few epochs 
early stops: if loss does not improve

correctness test:
?

# Training

Initialize Weights & Biases project for the project

Use k fold cross validation to avoid overfitting

1. Define experiement with different hyperparameters
2. Train model with train dataset split
3. Check model performance with validation dataset split
4. Log training run to Weights & Biases
5. Repeat

After all experiments have run select best runs hyperparameters for the final model.

# Evaluation

Metrics:
- F1 Score
- AOC
Averaging: ?
Error analysis:
- Confusion matrix

# Interpretation

Compare results of final model to expectations.
Run evaluation of final model with test dataset split.
But only at the very end, otherwise data leakage can happen.

Expectation: 70% accuracy with test dataset.